# Spikegram Save File  
This file will load the offline resonator data created in file:'parkinson_full_signal_analysis_8_bands'  
and create a spikegram from each file and save it to an output directory

In [1]:
import numpy as np

def spikegram_from_spikes_event(spikes_event, clk_freq, window_ms=None, time_s=None):
    ''' Create a Spectrogram from spikes event for a specified bins in miliseconds, default is 1 sec '''
    samples = spikes_event.size
    
    if time_s == None:
        time_s = samples // clk_freq # duration of signal in second rounded down
        
    if window_ms == None:
        window_ms = 1000   # Window size of 1 second
        
    num_bins = (time_s * 1000) // window_ms
    window_size = (clk_freq * window_ms) // 1000
    spikes_spectrogram = np.zeros(num_bins)
    for bin in range(num_bins):
        spikes_spectrogram[bin] = np.sum(spikes_event[bin*window_size: (1+bin)*window_size+1])
    
    return spikes_spectrogram

def create_spikegram(resonators_spikes_dict, clk_freq, window_ms=None, time_s=None):
    spikegram = {}
    time_bins = int(time_s) * 1000 // window_ms
    for axis, resonators_spikes in resonators_spikes_dict.items():
        spikegram[axis] = np.zeros((len(resonators_spikes), time_bins))   # for each axis: #resonators X #bins
        for n, spikes_event in enumerate(resonators_spikes.values()):
            spikegram[axis][n] = spikegram_from_spikes_event(spikes_event, clk_freq=clk_freq, window_ms=window_ms, time_s=int(time_s))
            spikegram[axis][n] -= np.mean(spikegram[axis][n])
            spikegram[axis][n][spikegram[axis][n] < 0] = 0
    return spikegram

In [2]:
import numpy as np

def align_fog_to_spikegram_high_sampling(fog_dict, spikegram_dict, spikegram_time_window_ms, clock_frequency):
    # Access the first spikegram in spikegram_dict
    first_key = list(spikegram_dict.keys())[0]  # Get the first key
    first_spikegram = spikegram_dict[first_key]  # Access the first spikegram

    # Get the number of time bins (x-axis length)
    spikegram_length = first_spikegram.shape[1]
    
    spikegram_time_window_ms = 10  # Each bin in spikegram_dict is 10 ms
    
    # Sampling rate for fog_dict
    clock_frequency = 15360
    fog_bin_size = int(spikegram_time_window_ms / (1000 / clock_frequency))  # Corresponding samples per spikegram bin (1536 samples per bin)
    
    # Create new fog_dict aligned with spikegram_dict
    aligned_fog_dict = {}
    for event_type, fog_samples in fog_dict.items():
        # Number of samples in fog_dict
        num_fog_samples = len(fog_samples)
        # Calculate the number of bins for fog_dict based on spikegram size
        aligned_fog = []
        
        for i in range(0, num_fog_samples, fog_bin_size):
            # Extract samples for this bin
            bin_samples = fog_samples[i:i+fog_bin_size]
            # Check if any sample in this bin is 1 (event occurred)
            aligned_fog.append(1 if np.any(bin_samples) else 0)
        
        # Truncate or pad to match the spikegram size
        aligned_fog = aligned_fog[:spikegram_length]  # Truncate to match spikegram size
        if len(aligned_fog) < spikegram_length:
            aligned_fog.extend([0] * (spikegram_length - len(aligned_fog)))  # Pad with 0s if shorter
        
        
        aligned_fog_dict[event_type] = np.array(aligned_fog)
    
    return aligned_fog_dict

In [3]:
import os
import pickle
from tqdm import tqdm

def save_spikegrams(offline_directory, output_directory, spikegram_time_window_ms, clock_frequency):
    # Create output directory to save there the spikegram offline
    os.makedirs(output_directory, exist_ok=True)
    
    # Iterate through all files in the directory
    with tqdm(total=len(os.listdir(offline_directory))) as pbar:
        for file in sorted(os.listdir(offline_directory)):
            if file.endswith('.pkl'):  # Ensure it's a grouped pickle file
                file_path = os.path.join(offline_directory, file)
                file_name = os.path.basename(file).split('.')[0]  # Extract base name
                
                # Load the data
                with open(file_path, 'rb') as f:
                    dictionaries = pickle.load(f)
                
                # Load relevant data
                resonators_spikes_dict = dictionaries['resonators_spikes_dict']
                time_s = dictionaries['time_s']
                fog_dict_r = dictionaries['fog_dict_r']
                
                # Create spikegram
                spikegram = create_spikegram(resonators_spikes_dict=resonators_spikes_dict,
                                            clk_freq=clock_frequency,
                                            window_ms=spikegram_time_window_ms,
                                            time_s=time_s)
                
                # Align FoG dict
                spikegram_fog_dict = align_fog_to_spikegram_high_sampling(fog_dict=fog_dict_r,
                                                                        spikegram_dict=spikegram,
                                                                        spikegram_time_window_ms=spikegram_time_window_ms,
                                                                        clock_frequency=clock_frequency)
                
                # Save each dictionary as a separate file
                save_dict = {
                    'spikegram_dict': spikegram,
                    'spikegram_fog_dict': spikegram_fog_dict
                }
                
                file_save_path = os.path.join(output_directory, f"{file_name}_spikegram.pkl")
                with open(file_save_path, 'wb') as f:
                    pickle.dump(save_dict, f)
                    
            pbar.update(1)   
            
    return


In [4]:
# folder_path = f'D:\\University\\project_data_files\\tdscfog_offline_data'
# output_directory = 'D:\\University\\project_data_files\\tdscfog_spikegram_offline_data'
spikegram_time_window_ms = 10
clock_frequency = 15360
'''
save_spikegrams(offline_directory=folder_path,
                output_directory=output_directory,
                spikegram_time_window_ms=spikegram_time_window_ms,
                clock_frequency=clock_frequency)
'''

100%|██████████| 411/411 [07:51<00:00,  1.15s/it]


Spikegram loading and verifying integrity of the files

In [5]:
import os
import pickle

def load_all_saved_data(offline_directory):
    # Define the offline data directory
    
    if not os.path.exists(offline_directory):
        raise FileNotFoundError(f"The directory {offline_directory} does not exist.")
    
    # Initialize a dictionary to store all loaded data
    all_data = {}
    
    # Iterate through all grouped .pkl files in the directory
    for file in sorted(os.listdir(offline_directory)):
        if file.endswith('.pkl'):  # Ensure it's a grouped pickle file
            file_path = os.path.join(offline_directory, file)
            file_name = os.path.basename(file).split('.')[0]  # Extract base name
            
            # Load the grouped data
            with open(file_path, 'rb') as f:
                grouped_data = pickle.load(f)
            
            # Add to the dictionary under the file name
            all_data[file_name] = grouped_data
    
    return all_data


In [6]:
folder_path = f'D:\\University\\tdscfog_spikegram_offline_data'
all_data = load_all_saved_data(folder_path)
flag = 0


for index, (file_name, data1) in enumerate(all_data.items()):
    spikegram_dict = data1['spikegram_dict']
    spikegram_fog_dict = data1['spikegram_fog_dict']
    for (spikegram_axis, spikegram_data), (fog_type, fog_event) in zip(spikegram_dict.items(), spikegram_fog_dict.items()):

        # print(spikegram_data.shape[1])
        # print(fog_event.shape[0])
        
        if spikegram_data.shape[1] != fog_event.shape[0]:
            print(f'Not the same size')
            flag = 1
    
if flag == 0:
    print(f'All files Spikegrams and Labels of same size')
            
        
        

All files Spikegrams and Labels of same size
